# PROYECTO - 3 FILTROS - INFORME

### David Alejandro Henao - Maria Camila Bravo

#### Objetivo: Implementar un flujo de procesamiento que incluya filtrado de las señales y extracción de características espectrales



## 1.ENTREGABLE

1. Consultar y explicar los dos métodos de reducción de ruido usados en el artículo: Robust LOESS y Non Local Means (10%), mostrar cómo se podrían implementar en Python (5%)

## 2.EXTRACIÓN DE REGISTROS 

De la base de datos extraer los registros que correspondan a bradicardia sinusal (SB Sinus Bradycardia) y fibrilación auricular (AFIB Atrial Fibrillation). Esta información está en el archivo Diagnostics.xlsx

# 3.CONSULTA

## 4. FLUJO DE PROCESAMIENTOS

Escoger 10 señales al azar y aplicar un flujo de procesamiento que conste de:

### FUJO 1 

### FUJO 2

### FUJO 3

Describir los resultados obtenidos y decidir si el resto del procesamiento se hace con el flujo 1, el flujo 2 o el flujo 3 (20%)

Con el flujo seleccionado procesar las señales de la base de datos ECGData.zip derivación II. En este punto deberíamos estar en el mismo punto donde partimos en el proyecto 2, con señales filtradas.

A las señales filtradas con el flujo seleccionado aplicar el procesamiento de normalización usado en el proyecto 2.

## 5. Para cada señal extraer la frecuencia que contiene la máxima potencia usando Welch (15%)

## 6. Crear una rutina que aplique sobre todos los archivos de la base de datos las rutina 3 al 5 y almacene los resultados en un dataframe donde se pueda registro, tipo de patología y el frecuencia de máxima potencia (fMP):

## 7.Comparar los resultados de fMP del proyecto 3 con los del proyecto 2 usando estadística descriptiva: gráficos y pruebas de hipótesis (15%) Discuta si hay más diferencias entre los tipos de señales con el flujo de procesamiento propuesto respecto al que se trabajó en el proyecto 2. 